<a href="https://colab.research.google.com/github/azubriichuk/Python-Project-A-B-testing/blob/main/Portfolio_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing libraries

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import ttest_ind
from google.colab import drive
import warnings

#SQL query

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import auth
auth.authenticate_user()
from google.cloud import bigquery
project_id = "data-analytics-mate"
client = bigquery.Client(project = "data-analytics-mate")

In [ ]:
query = """
WITH
  session_info AS (
    SELECT
      s.date,
      s.ga_session_id,
      sp.country,
      sp.device,
      sp.continent,
      sp.channel,
      ab.test,
      ab.test_group
    FROM `DA.ab_test` ab
    JOIN `DA.session` s
      ON ab.ga_session_id = s.ga_session_id
    JOIN `DA.session_params` sp
      ON ab.ga_session_id = sp.ga_session_id
  ),
  session_with_orders AS (
    SELECT
      session_info.date,
      session_info.country,
      session_info.device,
      session_info.continent,
      session_info.channel,
      session_info.test,
      session_info.test_group,
      COUNT(DISTINCT o.ga_session_id) AS session_with_orders
    FROM `DA.order` o
    JOIN session_info
      ON o.ga_session_id = session_info.ga_session_id
    GROUP BY
      session_info.date,
      session_info.country,
      session_info.device,
      session_info.continent,
      session_info.channel,
      session_info.test,
      session_info.test_group
  ),
  events AS (
    SELECT
      session_info.date,
      session_info.country,
      session_info.device,
      session_info.continent,
      session_info.channel,
      session_info.test,
      session_info.test_group,
      ep.event_name,
      COUNT(ep.ga_session_id) AS event_cnt
    FROM `DA.event_params` ep
    JOIN session_info
      ON ep.ga_session_id = session_info.ga_session_id
    GROUP BY
      session_info.date,
      session_info.country,
      session_info.device,
      session_info.continent,
      session_info.channel,
      session_info.test,
      session_info.test_group,
      ep.event_name
  ),
  sessions AS (
    SELECT
      session_info.date,
      session_info.country,
      session_info.device,
      session_info.continent,
      session_info.channel,
      session_info.test,
      session_info.test_group,
      COUNT(DISTINCT session_info.ga_session_id) AS session_cnt
    FROM session_info
    GROUP BY
      session_info.date,
      session_info.country,
      session_info.device,
      session_info.continent,
      session_info.channel,
      session_info.test,
      session_info.test_group
  ),
  account AS (
    SELECT
      session_info.date,
      session_info.country,
      session_info.device,
      session_info.continent,
      session_info.channel,
      session_info.test,
      session_info.test_group,
      COUNT(DISTINCT acs.ga_session_id) AS new_account_cnt
    FROM `DA.account_session` acs
    JOIN session_info
      ON acs.ga_session_id = session_info.ga_session_id
    GROUP BY
      session_info.date,
      session_info.country,
      session_info.device,
      session_info.continent,
      session_info.channel,
      session_info.test,
      session_info.test_group
  )
SELECT
  session_with_orders.date,
  session_with_orders.country,
  session_with_orders.device,
  session_with_orders.continent,
  session_with_orders.channel,
  session_with_orders.test,
  session_with_orders.test_group,
  'session_with_orders' AS event_name,
  session_with_orders.session_with_orders AS value
FROM session_with_orders
UNION ALL
SELECT
  events.date,
  events.country,
  events.device,
  events.continent,
  events.channel,
  events.test,
  events.test_group,
  event_name,
  event_cnt AS value
FROM events
UNION ALL
SELECT
  sessions.date,
  sessions.country,
  sessions.device,
  sessions.continent,
  sessions.channel,
  sessions.test,
  sessions.test_group,
  'sessions' AS event_name,
  session_cnt AS value
FROM sessions
UNION ALL
SELECT
  account.date,
  account.country,
  account.device,
  account.continent,
  account.channel,
  account.test,
  account.test_group,
  'new account' AS event_name,
  new_account_cnt AS value
FROM account
"""

df = client.query(query).to_dataframe()
df.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-12-08,Palestine,desktop,Asia,Direct,4,2,new account,1
1,2020-12-08,Palestine,desktop,Asia,Direct,3,2,new account,1
2,2020-11-06,Puerto Rico,desktop,Americas,Social Search,2,2,new account,1
3,2020-11-06,Puerto Rico,desktop,Americas,Social Search,1,1,new account,1
4,2020-12-08,Croatia,desktop,Europe,Direct,4,2,new account,1


#Preproccesing

In [ ]:
df['event_name'] = df['event_name'].replace('session_start', 'sessions')

df['event_name'] = df['event_name'].replace('new_accounts', 'new account')

#Metrics

In [ ]:
key_metrics = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new account']
event_denominator = 'sessions'

#Filtering data

In [ ]:
mask = (df['continent'] != '(not set)') & (df['event_name'].isin(key_metrics + [event_denominator]))
df_filtered = df[mask].copy()

#Grouping and summing value

In [ ]:
df_agg = df_filtered.groupby(['test', 'test_group', 'event_name'])['value'].sum().reset_index()

#Calculation

In [ ]:
def ab_test_improved(df):
    results = []

    # group by segments
    groups = df.groupby(['test'])

    for test_num, group_data in groups:

        # get denominator data (sessions)
        sessions = group_data[group_data['event_name'] == event_denominator]
        if sessions.empty:
            continue

        # get session counts for control (1) and test (2)
        control_den = sessions[sessions['test_group'] == 1]['value'].sum()
        test_den = sessions[sessions['test_group'] == 2]['value'].sum()

        if control_den == 0 or test_den == 0:
            continue

        # get list of metrics present in this segment
        available_metrics = [m for m in group_data['event_name'].unique() if m in key_metrics]

        for metric in available_metrics:
            metric_data = group_data[group_data['event_name'] == metric]

            control_event = metric_data[metric_data['test_group'] == 1]['value'].sum()
            test_event = metric_data[metric_data['test_group'] == 2]['value'].sum()

            # basic calculations
            conv_control = control_event / control_den
            conv_test = test_event / test_den

            # calc lift % (avoid division by zero)
            if conv_control > 0:
                metric_change = (conv_test - conv_control) / conv_control * 100
            else:
                metric_change = np.nan

            # z-test logic
            method = "skipped (n<30)"
            z_stat = np.nan
            p_value = np.nan
            significant = False

            if control_den > 30 and test_den > 30:
                try:
                    # calc z-stat and p-value
                    z_stat, p_value = sm.stats.proportions_ztest(
                        count=[test_event, control_event],
                        nobs=[test_den, control_den]
                    )
                    significant = p_value < 0.05
                    method = "z-test"
                except Exception:
                    method = "error"

            results.append({
                "test_number": test_num,
                "metric_full": f"{metric}/{event_denominator}",
                "numerator_metric": metric,
                "denominator_metric": event_denominator,

                "numerator_control": int(control_event),
                "denominator_control": int(control_den),
                "conversion_rate_control": round(conv_control, 4),

                "numerator_test": int(test_event),
                "denominator_test": int(test_den),
                "conversion_rate_test": round(conv_test, 4),

                "metric_change_%": round(metric_change, 3) if not np.isnan(metric_change) else None,
                "z_stat": round(float(z_stat), 4) if not np.isnan(z_stat) else None,
                "p_value": round(float(p_value), 5) if not np.isnan(p_value) else None,
                "significant": bool(significant),
                "method": method
            })

    return pd.DataFrame(results)

In [ ]:
final_df = ab_test_improved(df_agg)

In [ ]:
final_df.head()

,test_number,metric_full,numerator_metric,denominator_metric,numerator_control,denominator_control,conversion_rate_control,numerator_test,denominator_test,conversion_rate_test,metric_change_%,z_stat,p_value,significant,method
0,"(1,)",add_payment_info/sessions,add_payment_info,sessions,1981,91072,0.0218,2219,90639,0.0245,12.549,3.8719,0.00011,True,z-test
1,"(1,)",add_shipping_info/sessions,add_shipping_info,sessions,3030,91072,0.0333,3208,90639,0.0354,6.380,2.4849,0.01296,True,z-test
2,"(1,)",begin_checkout/sessions,begin_checkout,sessions,3780,91072,0.0415,3999,90639,0.0441,6.299,2.7528,0.00591,True,z-test
3,"(1,)",new account/sessions,new account,sessions,3816,91072,0.0419,3670,90639,0.0405,-3.367,-1.5128,0.13034,False,z-test
4,"(2,)",add_payment_info/sessions,add_payment_info,sessions,2333,101676,0.0229,2402,100790,0.0238,3.863,1.3194,0.18704,False,z-test


#Project Results & Conclusions
**1. Methodology: We automated the A/B test analysis using Python. The script calculated Conversion Rates and Lift (Metric Change %) for 4 key metrics:**


*   add_payment_info
*   add_shipping_info
*   begin_checkout
*   new_account


**2. Statistical Significance: We applied the Z-test for proportions to determine if the changes were statistically significant.**

*Confidence Level: 95%*

*Threshold: P-value < 0.05*

*Logic: If significant is True, the difference between the Control and Test groups is not due to chance.*

**3. Output: The results were saved to abtest_results.csv. This dataset contains granular data segmented by Test ID, Device, and Continent, and is ready for visualization in Tableau.**

**4. Next Steps:**

Import the CSV file into Tableau.

Visualize metrics.

Use the significant column to highlight reliable results.

#Saving the result

In [ ]:
drive.mount('/content/drive')
save_path = "/content/drive/MyDrive/Colab Notebooks/abtest.csv"
final_df.to_csv(save_path, index=False)
print(f"\nFile saved at: {save_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

File saved at: /content/drive/MyDrive/Colab Notebooks/abtest.csv


#Tableau result visualization:
https://public.tableau.com/app/profile/artemii.zubriichuk/viz/ProjectABTesting/Dashboard1

#Download the result:
https://drive.google.com/file/d/1yqlk4RMnFZtmxgrE-Pgik0qbYGZH6ksW/view?usp=sharing